# 日本語 BERT Base Model Fine-tuning & Deployment on Inferentia2/Trainium
本 Notebook の元ネタのブログはこちらから
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part1/
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part2/

## 事前準備
本 notebook　は Neuron 2.13.2 環境下で動作確認しています

In [1]:
!pip install -U pip
!pip install -U transformers[ja]==4.27.4 datasets
!pip uninstall -y wandb # if installed, needs to setup wandb

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
!pip list | grep "neuron\|torch\|transformers"

aws-neuronx-runtime-discovery 2.9
libneuronxla                  0.5.440
neuronx-cc                    2.9.0.40+07376825f
neuronx-distributed           0.3.0
neuronx-hwm                   2.9.0.2+f79d59e7b
pytorch-lightning             1.8.6
sentence-transformers         2.2.2
torch                         1.13.1
torch-neuronx                 1.13.1.1.10.1
torch-xla                     1.13.1+torchneurona
torchmetrics                  0.10.3
torchvision                   0.14.1
transformers                  4.27.4
transformers-neuronx          0.6.106


In [3]:
!dpkg --list | grep neuron

ii  aws-neuronx-collectives                2.16.16.0-e59c7bb3e               amd64        neuron_ccom built using CMake
ii  aws-neuronx-dkms                       2.12.18.0                         amd64        aws-neuronx driver in DKMS format.
hi  aws-neuronx-oci-hook                   2.2.16.0                          amd64        neuron_oci_hook built using CMake
ii  aws-neuronx-runtime-lib                2.16.14.0-61fdc395f               amd64        neuron_runtime built using CMake
ii  aws-neuronx-tools                      2.13.4.0                          amd64        Neuron profile and debug tools


In [4]:
!sudo rmmod neuron; sudo modprobe neuron

## データセットの準備
本テストでは、Huggingface Hub で利用可能な以下のセンチメント（感情）データセットのうち、日本語のサブセットを使用します。
https://huggingface.co/datasets/tyqiangz/multilingual-sentiments

本テストではテキストデータをPositiveかNegativeに分類する 2 クラスの分類問題として扱うことにします。元々のデータセットは positive(LABEL_0)、neutral(LABEL_1)、negative(LABEL_2)としてラベル付けされていますが、neutralのデータは使用しないこととし、ラベルをpositive(LABEL_0)、negative(LABEL_1)として再定義します。

In [5]:
from transformers import BertJapaneseTokenizer
from datasets import load_dataset

# Prepare dataset
dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")
dataset = dataset.remove_columns(["source"])
dataset = dataset.filter(lambda dataset: dataset["label"] != 1)
dataset = dataset.map(lambda dataset: {"labels": int(dataset["label"] == 2)}, remove_columns=["label"])

print(dataset["train"][20000])
print(dataset["train"][50000])

{'text': '箱を開けただけで予想以上に匂いが甘くて 私には合わないかもです。 でも もったいないので今使ってるのが無くなったら使ってみます。', 'labels': 1}
{'text': '色々な変換アダプが売ってますけど長さも結構ありますし重宝しています。主にタブレットから→テレビに使ってるんですけど子供達に動画を見せる時タブレットを直接見せるよりテレビで大きな画面で見せた方が良いですね。なんの設定も要らないので直接繋げればすぐ使える所も気に入っています。', 'labels': 0}


次に、文章テキストのままだとモデルのトレーニングはできないため、テキストを意味のある単位で分割（トークナイズ）した上で数値に変換します。トークナイザーには MeCab ベースの BertJapaneseTokenizer を利用しました。

In [6]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

train_dataset = tokenized_datasets["train"].shuffle().select(range(4000))
eval_dataset = tokenized_datasets["test"].shuffle().select(range(256))

# Save dataset
train_dataset.save_to_disk("./train/")
eval_dataset.save_to_disk("./test/")

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/256 [00:00<?, ? examples/s]

実際にどのように変換されているのか、以下のスクリプトを実行し確認してみましょう。

In [7]:
index = 20000
print(dataset["train"][index])
print('Tokenize:', tokenizer.tokenize(dataset["train"]['text'][index]))
print('Encode:', tokenizer.encode(dataset["train"]['text'][index]))

{'text': '箱を開けただけで予想以上に匂いが甘くて 私には合わないかもです。 でも もったいないので今使ってるのが無くなったら使ってみます。', 'labels': 1}
Tokenize: ['箱', 'を', '開け', 'た', 'だけ', 'で', '予想', '以上', 'に', '匂い', 'が', '甘', '##く', 'て', '私', 'に', 'は', '合わ', 'ない', 'かも', 'です', '。', 'でも', 'もっ', '##たい', '##ない', 'ので', '今', '使っ', 'てる', 'の', 'が', '無くなっ', 'たら', '使っ', 'て', 'み', 'ます', '。']
Encode: [2, 3684, 11, 9709, 10, 687, 12, 4663, 695, 7, 23071, 14, 5063, 28504, 16, 1325, 7, 9, 13964, 80, 4830, 2992, 8, 962, 1750, 4541, 3721, 947, 744, 2110, 7134, 5, 14, 15446, 3318, 2110, 16, 546, 2610, 8, 3]


## Trainer API を使用した トレーニング（ファインチューニング）実行
Transformers には Trainer という便利なクラスがあり、Torch Neuron からも利用可能です。 ここでは Trainer API を利用してトレーニングを実行していきます。

### neuron_parallel_compile による事前コンパイル
トレーニングの各ステップでは、グラフがトレースされ、トレースされたグラフが以前のものと異なる場合は、再度計算グラフのコンパイルが発生します。大規模なモデルの場合、各グラフのコンパイル時間が長くなることがあり、トレーニング時間の中で占めるコンパイル時間がボトルネックとなってしまう場合もあり得ます。このコンパイル時間を短縮するため、PyTorch Neuron では neuron_parallel_compile ユーティリティが提供されています。neuron_parallel_compile は、スクリプトの試行からグラフを抽出し並列事前コンパイルを実施、コンパイル結果（NEFF : Neuron Executable File Format）をキャッシュとしてディスク上に保持します。

では実際に事前コンパイルを実行してみましょう。以下の内容でbert-jp-precompile.pyというファイル名の Python スクリプトを作成し実行します。スクリプトは基本的にこの後実行するトレーニングスクリプトと同じ内容ですが、neuron_parallel_compileはグラフの高速コンパイルのみを目的とし実際の演算は実行されず、出力結果は無効となります。トレーニング実行中も必要に応じてグラフはコンパイルされるため、この事前コンパイルのプロセスはスキップし、次のトレーニング処理に直接進んでも問題はありません。

コンパイル時間を短縮するためデータセット、epoch 数を制限している点にご注意ください。コンパイル結果は `/var/tmp/neuron-compile-cache/` 以下に保存されています。

In [8]:
%%writefile bert-jp-precompile.py

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))

training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()

eval_result = trainer.evaluate()

Overwriting bert-jp-precompile.py


環境変数`XLA_USE_BF16=1`　を設定することで、BF16 + Stochastic Rounding で実行します。

In [9]:
!time XLA_USE_BF16=1 neuron_parallel_compile python3 bert-jp-precompile.py

2023-09-05 08:41:16.000784:  3606836  INFO ||PARALLEL_COMPILE||: Removing existing workdir /tmp/parallel_compile_workdir
2023-09-05 08:41:16.000788:  3606836  INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model

### シングルワーカーでのトレーニング実行
次に実際にトレーニングを実行してみます。事前コンパイルを実行した場合でも、追加のコンパイルが発生することがあります。一通りのコンパイルが終了した後、2度目以降の実行では、Neuron コアの恩恵を受けた高速トレーニングを体験できます。以下の内容で bert-jp-single.py というファイル名の Python スクリプトを作成し実行してみましょう。

先程の事前コンパイルとは異なり、今回は実際にトレーニングを実行するため、用意したデータセット全てに対して epoch = 10 で実行します。

In [10]:
%%writefile bert-jp-single.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

Overwriting bert-jp-single.py


In [11]:
!time XLA_USE_BF16=1 python3 bert-jp-single.py

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

ステップ数 5000 のトレーニングが 9~10分程で完了しました。

トレーニング実行中に、AWS Neuron で提供される `neuron-top` ツールを利用すると、Neuron コア及び vCPU の利用率、アクセラレータメモリ、ホストメモリの利用状況等を確認することができます。inf2.xlarge には、一つの Inferentia2 チップ、チップ内に二つの Neuron コアが搭載されています。結果から、二つある Neuron コア（NC0 及び NC1）のうち一つの Neuron コアのみが利用されていることが分かります。まだ最適化の余地はありそうです。

生成されたモデルから期待通りの出力が得られるか確認しておきましょう。

In [12]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "./results/")

print(classifier("大変すばらしい商品でした。感激です。"))
print(classifier("期待していた商品とは異なりました。残念です。"))

[{'label': 'LABEL_0', 'score': 0.9999337196350098}]
[{'label': 'LABEL_1', 'score': 0.9999854564666748}]


期待通りの出力を得られることが確認できたようです。

## torchrun を用いたマルチワーカーでのトレーニング実行
それでは、先程のトレーニングスクリプトに変更を加え、二つある Neuron コアを有効活用してみましょう。複数の Neuron コアを利用したマルチワーカーで実行するためには `torchrun` コマンドを利用します。`torchrun` コマンドに対して、オプション `--nproc_per_node` で利用する Neuron コアの数（並列実行するワーカー数）を指定します。trn1.2xlarge (inf2.xlarge) では 2 を、trn1.32xlargeの場合は 2, 8, 32 が指定可能です。

`torchrun` を利用したデータパラレルトレーニングを実行するにあたって、先程のスクリプトに一部変更を加えた `bert-jp-dual.py` というファイル名のスクリプトを作成し実行します。

それでは変更後のスクリプトを利用して　inf2.xlarge 上の二つ Neuron コアを利用したトレーニングを実行してみましょう。シングルワーカーでのトレーニング結果と比較し `Total train batch size` の値が倍の 16 に、`Total optimization steps` が半分の 2500 となっている点を確認できると思います。

シングルワーカー時の手順同様、まずは事前コンパイルを実行し、その後に実際のトレーニングを実行します。

In [13]:
%%writefile bert-jp-dual-precompile.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))


training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()

eval_result = trainer.evaluate()

Overwriting bert-jp-dual-precompile.py


In [14]:
!time XLA_USE_BF16=1 neuron_parallel_compile torchrun --nproc_per_node=2 bert-jp-dual-precompile.py

2023-09-05 08:55:01.000915:  3711014  INFO ||PARALLEL_COMPILE||: Removing existing workdir /tmp/parallel_compile_workdir
2023-09-05 08:55:01.000919:  3711014  INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.Laye

In [15]:
%%writefile bert-jp-dual.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

Overwriting bert-jp-dual.py


In [16]:
!time XLA_USE_BF16=1 torchrun --nproc_per_node=2 bert-jp-dual.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

トレーニング実行中の neuron-top の出力も確認してみましょう。今度は二つの Neuron コアが利用されている事が確認できると思います。トレーニングに要する実行時間も 5~6 分に削減されました。

## 推論実行
先ほどは生成されたモデルから期待通りの出力が得られるかどうかCPU上で推論実行し、結果を確認しました。ここでは生成されたモデルをinf2.xlarge上で推論実行します。

In [17]:
import torch
import torch_neuronx
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import transformers

from transformers import BertForSequenceClassification, BertJapaneseTokenizer

def encode(tokenizer, *inputs, max_length=128, batch_size=1):
    tokens = tokenizer.encode_plus(
        *inputs,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    return (
        torch.repeat_interleave(tokens['input_ids'], batch_size, 0),
        torch.repeat_interleave(tokens['attention_mask'], batch_size, 0),
        torch.repeat_interleave(tokens['token_type_ids'], batch_size, 0),
    )

tokenizer = AutoTokenizer.from_pretrained("./results/")
model = AutoModelForSequenceClassification.from_pretrained("./results/", torchscript=True)


sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

CPU paraphrase logits: [[ 4.7050467 -4.1804614]]
CPU paraphrase logits: [[-4.7470703  4.467029 ]]


### Compile the model for Neuron

In [18]:
filename = 'model.pt'

In [19]:
model_neuron = torch_neuronx.trace(model, paraphrase)

# Save the TorchScript for inference deployment
torch.jit.save(model_neuron, filename)

2023-09-05T09:05:21Z Running DoNothing
2023-09-05T09:05:21Z DoNothing finished after 0.000 seconds
2023-09-05T09:05:21Z Running CanonicalizeIR
2023-09-05T09:05:21Z CanonicalizeIR finished after 0.020 seconds
2023-09-05T09:05:21Z Running ExpandBatchNorm
2023-09-05T09:05:21Z ExpandBatchNorm finished after 0.021 seconds
2023-09-05T09:05:21Z Running ResolveComplicatePredicates
2023-09-05T09:05:22Z ResolveComplicatePredicates finished after 0.016 seconds
2023-09-05T09:05:22Z Running AffinePredicateResolution
2023-09-05T09:05:22Z AffinePredicateResolution finished after 0.018 seconds
2023-09-05T09:05:22Z Running EliminateDivs
2023-09-05T09:05:22Z EliminateDivs finished after 0.018 seconds
2023-09-05T09:05:22Z Running PerfectLoopNest
2023-09-05T09:05:22Z PerfectLoopNest finished after 0.017 seconds
2023-09-05T09:05:22Z Running Simplifier
2023-09-05T09:05:22Z Simplifier finished after 0.238 seconds
2023-09-05T09:05:22Z Running GenericAccessSimplifier
2023-09-05T09:05:22Z GenericAccessSimplifie

### Run inference and compare results

In [20]:
model_neuron = torch.jit.load(filename)

sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

Neuron paraphrase logits: [[ 4.7064433 -4.1799984]]
Neuron paraphrase logits: [[-4.7476482  4.4669704]]


CPUで推論実行した結果と同様の結果が得られている事が確認できました。推論性能を評価する方法は以下のサンプルをご参照下さい。
+ https://github.com/aws-neuron/aws-neuron-sdk/blob/master/src/examples/pytorch/torch-neuronx/bert-base-cased-finetuned-mrpc-inference-on-trn1-tutorial.ipynb